In [13]:
import pylab

pylab.figure(1)
pylab.plot([1,2,3,4], [1,7,3,5], 'ro', linewidth=30)
#pylab.show()
pylab.xlabel('Label 1', fontsize=14)
pylab.ylabel('Label 2', fontsize=18)
pylab.title('My figure', fontsize=12)
pylab.savefig('Fig1')


In [14]:
from os.path import join
from codecs import open
import pylab
import numpy as np
HOME = '/home/roger/Workspace/edX/MITx/6.00.2x/data'

In [12]:
# L1 PROBLEM 3  (3 points possible)
# In this problem and the next, we will explore how to make sense of a collection of data by first parsing the 
# data, and then producing a meaningful plot that visually explains what the raw numbers say.
# 
# The file julyTemps.txt contains the daily maximum and minimum temperatures for each day in Boston for the 31 days 
# of July 2012. Open the file in a text editor and look at how it is formatted. The goal of this exercise is to 
# produce a plot that shows the difference between the high and the low temperature for each day.
# 
# First, we will write a function to load the data out of this file (for a review of file loading and processing, 
# see Problem Set 1 - Review of IO. You may also wish to read up on the Python docs on file objects, particularly 
# how to read multiple lines in a file).
# 
# We want this function to return a tuple of 2 lists: one for high temperatures and one for low temperatures.
# 
# The next set of questions will help guide you in writing a function that opens the datafile and parses the data 
# into Python data structures.

def loadFile():
    inFile = open(join(HOME, 'julyTemps.txt'), 'r', 'utf-8')
    high = []
    low = []
    for line in inFile:
        fields = line.split()
        if len(fields) != 3 or 'Boston' == fields[0] or 'Day' == fields[0]:
            continue
        else:
            high.append(int(fields[1]))
            low.append(int(fields[2]))
    return (low, high)

(low, high) = loadFile() 

In [14]:
# L1 PROBLEM 4  (4 points possible)
# Note: This problem is contingent upon completing the exercises from L1 Problem 3.
# 
# Now for the plotting function! In pylab, the plot function takes in two equal sized lists and uses the first list 
# as a list of the x-coordinates and the second list as a list of the y-coordinates.
# 
# In this problem we'll build a function producePlot(lowTemps, highTemps) which takes as parameters lowTemps (the 
# list of low temperatures from your previous function) and highTemps (the list of high temperatures from your 
# previous function).

def producePlot(lowTemps, highTemps):
    pylab.clf()
    diffTemps = list(np.array(highTemps) - np.array(lowTemps))
    pylab.plot(range(1,32), diffTemps)
    pylab.title('Day by Day Ranges in Temperature in Boston in July 2012')
    pylab.xlabel('Days')
    pylab.ylabel('Temperature Ranges')
    pylab.show()
    
producePlot(low, high)

In [15]:
### Morgages problem - Plot
import pylab

#set line width
pylab.rcParams['lines.linewidth'] = 6
#set font size for titles 
pylab.rcParams['axes.titlesize'] = 20
#set font size for labels on axes
pylab.rcParams['axes.labelsize'] = 20
#set size of numbers on x-axis
pylab.rcParams['xtick.major.size'] = 5
#set size of numbers on y-axis
pylab.rcParams['ytick.major.size'] = 5

def findPayment(loan, r, m):
    """Assumes: loan and r are floats, m an int
    Returns the monthly payment for a mortgage of size
    loan at a monthly rate of r for m months"""
    return loan*((r*(1+r)**m)/((1+r)**m - 1))

class MortgagePlots(object):
    
    def plotPayments(self, style):
        pylab.plot(self.paid[1:], style, label = self.legend)
        
    def plotTotPd(self, style):
        totPd = [self.paid[0]]
        for i in range(1, len(self.paid)):
            totPd.append(totPd[-1] + self.paid[i])
        pylab.plot(totPd, style, label = self.legend)


class Mortgage(MortgagePlots, object):
    """Abstract class for building different kinds of mortgages"""
    def __init__(self, loan, annRate, months):
        """Create a new mortgage"""
        self.loan = loan
        self.rate = annRate/12.0
        self.months = months
        self.paid = [0.0]
        self.owed = [loan]
        self.payment = findPayment(loan, self.rate, months)
        self.legend = None #description of mortgage

    def makePayment(self):
        """Make a payment"""
        self.paid.append(self.payment)
        reduction = self.payment - self.owed[-1]*self.rate
        self.owed.append(self.owed[-1] - reduction)

    def getTotalPaid(self):
        """Return the total amount paid so far"""
        return sum(self.paid)

    def __str__(self):
        return self.legend

    
# fixed-rate mortgage
class Fixed(Mortgage):
    def __init__(self, loan, r, months):
        Mortgage.__init__(self, loan, r, months)
        self.legend = 'Fixed, ' + str(r*100) + '%'

# fixed-rate mortgage with up-front points
class FixedWithPts(Fixed):
    def __init__(self, loan, r, months, pts):
        Fixed.__init__(self, loan, r, months)
        self.pts = pts
        self.paid = [loan*(pts/100.0)]
        self.legend += ', ' + str(pts) + ' points'

# mortgage that changes interest rate after 48 months
class TwoRate(Mortgage):
    def __init__(self,loan,r,months,teaserRate,teaserMonths):
        Mortgage.__init__(self, loan, teaserRate, months)
        self.teaserMonths = teaserMonths
        self.teaserRate = teaserRate
        self.nextRate = r/12.0
        self.legend = str(teaserRate*100)\
            + '% for ' + str(self.teaserMonths)\
            + ' months, then ' + str(r*100) + '%'

    def makePayment(self):
        if len(self.paid) == self.teaserMonths + 1:
            self.rate = self.nextRate
            self.payment = findPayment(self.owed[-1], self.rate,
                                       self.months - \
                                       self.teaserMonths)
        Mortgage.makePayment(self)


class MortgagePlots(object):
    
    def plotPayments(self, style):
        pylab.plot(self.paid[1:], style, label = self.legend)
        
    def plotTotPd(self, style):
        totPd = [self.paid[0]]
        for i in range(1, len(self.paid)):
            totPd.append(totPd[-1] + self.paid[i])
        pylab.plot(totPd, style, label = self.legend)


def plotMortgages(morts, amt):
    styles = ['b-', 'r-.', 'g:']
    payments = 0 #number to identify a figure
    cost = 1 #number to identify a figure
    pylab.figure(payments)
    pylab.title('Monthly Payments of Different $' + str(amt)
                + ' Mortgages')
    pylab.xlabel('Months')
    pylab.ylabel('Monthly Payments')
    pylab.figure(cost)
    pylab.title('Cost of Different $' + str(amt) + ' Mortgages')
    pylab.xlabel('Months')
    pylab.ylabel('Total Payments')
    for i in range(len(morts)):
        pylab.figure(payments)
        morts[i].plotPayments(styles[i])
        pylab.figure(cost)
        morts[i].plotTotPd(styles[i])
    pylab.figure(payments)
    pylab.legend(loc = 'upper center')
    pylab.figure(cost)
    pylab.legend(loc = 'best')


def compareMortgages(amt, years, fixedRate, pts, ptsRate,
                    varRate1, varRate2, varMonths):
    totMonths = years*12
    fixed1 = Fixed(amt, fixedRate, totMonths)
    fixed2 = FixedWithPts(amt, ptsRate, totMonths, pts)
    twoRate = TwoRate(amt, varRate2, totMonths,
                      varRate1, varMonths)
    morts = [fixed1, fixed2, twoRate]
    for m in range(totMonths):
        for mort in morts:
            mort.makePayment()
    plotMortgages(morts, amt)


compareMortgages(amt=200000, years=30, fixedRate=0.07,
                 pts = 3.25, ptsRate=0.05, varRate1=0.045,
                 varRate2=0.095, varMonths=48)
pylab.show()

In [11]:
# RANDOM DRUNKS Algorithm

class Location(object):
    def __init__(self, x, y):
        """x and y are floats"""
        self.x = x
        self.y = y
        
    def move(self, deltaX, deltaY):
        """deltaX and deltaY are floats"""
        return Location(self.x + deltaX, self.y + deltaY)
    
    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    def distFrom(self, other):
        ox = other.x
        oy = other.y
        xDist = self.x - ox
        yDist = self.y - oy
        return (xDist**2 + yDist**2)**0.5
    
    def __str__(self):
        return '<' + str(self.x) + ', ' + str(self.y) + '>'
    
    
class Field(object):
    
    def __init__(self):
        self.drunks = {}
        
    def addDrunk(self, drunk, loc):
        if drunk in self.drunks:
            raise ValueError('Duplicate drunk')
        else:
            self.drunks[drunk] = loc
            
    def moveDrunk(self, drunk):
        if not drunk in self.drunks:
            raise ValueError('Drunk not in field')
        xDist, yDist = drunk.takeStep()
        currentLocation = self.drunks[drunk]
        #use move method of Location to get new location
        self.drunks[drunk] = currentLocation.move(xDist, yDist)
        
    def getLoc(self, drunk):
        if not drunk in self.drunks:
            raise ValueError('Drunk not in field')
        return self.drunks[drunk]


import random

class Drunk(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'This drunk is named ' + self.name
    

class UsualDrunk(Drunk):
    def takeStep(self):
        stepChoices =\
            [(0.0,1.0), (0.0,-1.0), (1.0, 0.0), (-1.0, 0.0)]
        return random.choice(stepChoices)


def walk(f, d, numSteps):
    start = f.getLoc(d)
    for s in range(numSteps):
        f.moveDrunk(d)
    return(start.distFrom(f.getLoc(d)))


def simWalks(numSteps, numTrials):
    homer = UsualDrunk('Homer')
    origin = Location(0, 0)
    distances = []
    for t in range(numTrials):
        f = Field()
        f.addDrunk(homer, origin)
        distances.append(walk(f, homer, numSteps))
    return distances


def drunkTest(numTrials = 20):
    for numSteps in [10, 100, 1000, 10000]:
        distances = simWalks(numSteps, numTrials)
        print 'Random walk of ' + str(numSteps) + ' steps'
        print ' Mean =', sum(distances)/len(distances)
        print ' Max =', max(distances), 'Min =', min(distances)

drunkTest()

## import pylab

## #set line width
## pylab.rcParams['lines.linewidth'] = 6
## #set font size for titles
## pylab.rcParams['axes.titlesize'] = 20
## #set font size for labels on axes
## pylab.rcParams['axes.labelsize'] = 20
## #set size of numbers on x-axis
## pylab.rcParams['xtick.major.size'] = 5
## #set size of numbers on y-axis
## pylab.rcParams['ytick.major.size'] = 5
## #set size of markers
## pylab.rcParams['lines.markersize'] = 10

Random walk of 10 steps
 Mean = 3.39687037029
 Max = 5.65685424949 Min = 1.41421356237
Random walk of 100 steps
 Mean = 7.59737354706
 Max = 15.0332963784 Min = 1.41421356237
Random walk of 1000 steps
 Mean = 35.1106499902
 Max = 80.6597793203 Min = 7.21110255093
Random walk of 10000 steps
 Mean = 102.287322841
 Max = 274.415742989 Min = 7.07106781187


In [16]:
# Plotting drunkTest

def drunkTestP(numTrials = 50):
    stepsTaken = [10, 100, 1000, 10000]
    meanDistances = []
    for numSteps in stepsTaken:
        distances = simWalks(numSteps, numTrials)
        meanDistances.append(sum(distances)/len(distances))
    pylab.clf()
    pylab.plot(stepsTaken, meanDistances)
    pylab.title('Mean Distance from Origin')
    pylab.xlabel('Steps Taken')
    pylab.ylabel('Steps from Origin')
    pylab.show()

drunkTestP()

In [18]:
# Plotting square roots

def drunkTestP1(numTrials = 50):
    stepsTaken = [10, 100, 1000, 10000]
    meanDistances = []
    squareRootOfSteps = []
    for numSteps in stepsTaken:
        distances = simWalks(numSteps, numTrials)
        meanDistances.append(sum(distances)/len(distances))
        squareRootOfSteps.append(numSteps**0.5)
    pylab.clf()
    pylab.plot(stepsTaken, meanDistances, 'b-',
               label = 'Mean distance')
    pylab.plot(stepsTaken, squareRootOfSteps, 'g-.',
               label = 'Square root of steps')
    pylab.title('Mean Distance from Origin')
    pylab.xlabel('Steps Taken')
    pylab.ylabel('Steps from Origin')
    pylab.legend()
    pylab.show()
    
drunkTestP1()

In [19]:
#Look at different kinds of drunks

class UsualDrunk(Drunk):
    def takeStep(self):
        stepChoices =\
            [(0.0,1.0), (0.0,-1.0), (1.0, 0.0), (-1.0, 0.0)]
        return random.choice(stepChoices)

class ColdDrunk(Drunk):
    def takeStep(self):
        stepChoices =\
            [(0.0,0.95), (0.0,-1.0), (1.0, 0.0), (-1.0, 0.0)]
        return random.choice(stepChoices)

class EDrunk(Drunk):
    def takeStep(self):
        deltaX = random.random()
        if random.random() < 0.5:
            deltaX = -deltaX
        deltaY = random.random()
        if random.random() < 0.5:
            deltaY = -deltaY
        return (deltaX, deltaY)

# New version of simWalks

def simWalks(numSteps, numTrials, dClass):
    homer = dClass('Homer')
    origin = Location(0, 0)
    distances = []
    for t in range(numTrials):
        f = Field()
        f.addDrunk(homer, origin)
        distances.append(walk(f, homer, numSteps))
    return distances


def drunkTestP0(numTrials = 50):
    stepsTaken = [10, 100, 1000, 10000]
    for dClass in (UsualDrunk, ColdDrunk, EDrunk):
        meanDistances = []
        for numSteps in stepsTaken:
            distances = simWalks(numSteps, numTrials, dClass)
            meanDistances.append(sum(distances)/len(distances))
        pylab.plot(stepsTaken, meanDistances,
                   label = dClass.__name__)
        pylab.title('Mean Distance from Origin')
        pylab.xlabel('Steps Taken')
        pylab.ylabel('Steps from Origin')
        pylab.legend(loc = 'upper left')
    pylab.show()

drunkTestP0()

In [6]:
import random

# Code Sample A
mylist = []

for i in xrange(random.randint(1, 10)):
    random.seed(0) #randint(1,10) -> 9, 8, 5, 3, 6, 5...
    k = random.randint(1, 10)
    if k > 3:
        number = random.randint(1, 10)
        if number not in mylist:
            mylist.append(number)
print mylist

[8]


In [ ]:
# L2 PROBLEM 2  (5 points possible)
# Using the random module
# 
# This problem asks you to write a short function that uses the the random module. The random module has many 
# useful functions - play around with them in your interpreter to see how much you can do! 
# To test this code yourself, put the line import random at the top of your code file, to import all of the 
# functions in the random module. To call random module methods, preface them with random., as in this sample 
# interpreter session:
# 
# >>> import random
# >>> random.randint(1, 5)
# 4
#
# >>> random.choice(['apple', 'banana', 'cat'])
# 'cat'
#
# How would you randomly generate an even number x, 0 <= x < 100? Fill out the definition for the function 
# genEven(). 
# Please generate a uniform distribution over the even numbers between 0 and 100 (not including 100).

import random
def genEven():
    '''
    Returns a random even number x, where 0 <= x < 100
    '''
    return random.choice(range(0,100,2))

for _ in range(100):
    print genEven(),

In [ ]:
# L2 PROBLEM 3A  (5 points possible)
# Write a deterministic program, deterministicNumber, that returns an even number between 9 and 21.

import random
def deterministicNumber():
    '''
    Deterministically generates and returns an even number between 9 and 21
    '''
    random.seed(0)  # use a constant seed
    return 2 * random.randint(5, 10)

for _ in range(10):
    print deterministicNumber(),

In [ ]:
# L2 PROBLEM 3B  (5 points possible)
# Write a uniformly distributed stochastic program, stochasticNumber, that returns an even number between 9 and 21.

import random
def stochasticNumber():
    '''
    Stochastically generates and returns a uniformly distributed even number between 9 and 21
    '''
    return random.randrange(10, 22, 2)
    
for _ in range(10):
    print stochasticNumber(),